In [1]:
data_source = ['math-amc', 'math-aime', 'gpqa', 'mip-insufficient', 'mip-question', 'bbh', 'bbh_boolean_expressions', 'bbh_causal_judgement', 'bbh_date_understanding', 'bbh_disambiguation_qa', 'bbh_logical_deduction_five_objects', 'bbh_movie_recommendation', 'bbh_navigate', 'bbh_object_counting', 'bbh_snarks', 'underthink-bench', ]

In [10]:
import os

os.environ['HF_HOME'] = '/nas-ssd2/joykirat/.cache/huggingface'
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
from transformers import AutoTokenizer, AutoModelForCausalLM
from vllm import LLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B")
# Qwen/Qwen3-4B
# deepseek-ai/DeepSeek-R1-Distill-Llama-8B
# deepseek-ai/DeepSeek-R1-Distill-Qwen-7B
# model = LLM(model="Qwen/Qwen3-4B", gpu_memory_utilization=0.95, tensor_parallel_size=1, task="embed")





INFO 09-23 11:27:38 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 09-23 11:27:39 [__init__.py:239] Automatically detected platform cuda.


In [ ]:
# ## SUPERGPQA
# BaseModelPath='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/eval/verlCheckpoint/mathbaseRun/qwen4b_10k_base_model/rollout/val_0_v6.jsonl'
# CorrectnessOnlyPath='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/train/verlCheckpoint/CorrectnessOnly/qwen4b_dapo_math_10k_correctnessOnly/rollout/val_140_v6.jsonl'
# no_summary_path='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/train/verlCheckpoint/NonSummary/qwen4b_dapo_math_10k_context_linear_reward_no_summary_no_difficulty/rollout/val_140_v5.jsonl'
# ourMethodPath='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/train/verlCheckpoint/Summary/qwen4b_dapo_math_10k_context_linear_reward_with_summary_attention/rollout/val_140_v5.jsonl'

## GPQA
# BaseModelPath='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/eval/verlCheckpoint/mathbaseRun/qwen4b_10k_base_model/rollout/val_0_v0.jsonl'
# CorrectnessOnlyPath='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/train/verlCheckpoint/CorrectnessOnly/qwen4b_dapo_math_10k_correctnessOnly/rollout/val_140_v0.jsonl'
# no_summary_path='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/train/verlCheckpoint/NonSummary/qwen4b_dapo_math_10k_context_linear_reward_no_summary_no_difficulty/rollout/val_140_v0.jsonl'
# ourMethodPath='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/train/verlCheckpoint/Summary/qwen4b_dapo_math_10k_context_linear_reward_with_summary_attention/rollout/val_140_v0.jsonl'


## BBEH
BaseModelPath='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/eval/verlCheckpoint/mathbaseRun/qwen4b_10k_base_model/rollout/val_0_v5.jsonl'
CorrectnessOnlyPath='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/train/verlCheckpoint/CorrectnessOnly/qwen4b_dapo_math_10k_correctnessOnly/rollout/val_140_v5.jsonl'
no_summary_path='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/train/verlCheckpoint/NonSummary/qwen4b_dapo_math_10k_context_linear_reward_no_summary_no_difficulty/rollout/val_140_v4.jsonl'
ourMethodPath='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/train/verlCheckpoint/Summary/qwen4b_dapo_math_10k_context_linear_reward_with_summary_attention/rollout/val_140_v4.jsonl'
adapThinkPath='/nas-ssd2/joykirat/code/verl-fork/baselines/AdaptThink/bbeh_test_dataset/test_generation_AdaptThink-custom-qwen4b_delta0_05.parquet'

## BBH
# BaseModelPath='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/eval/verlCheckpoint/mathbaseRun/qwen4b_10k_base_model/rollout/val_0_v2.jsonl'
# CorrectnessOnlyPath='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/train/verlCheckpoint/CorrectnessOnly/qwen4b_dapo_math_10k_correctnessOnly/rollout/val_140_v2.jsonl'
# no_summary_path='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/train/verlCheckpoint/NonSummary/qwen4b_dapo_math_10k_context_linear_reward_no_summary_no_difficulty/rollout/val_140_v1.jsonl'
# ourMethodPath='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/train/verlCheckpoint/Summary/qwen4b_dapo_math_10k_context_linear_reward_with_summary_attention/rollout/val_140_v2.jsonl'

In [2]:
import json
Baseline_model_data = []
with open(BaseModelPath, 'r') as f:
    for line in f:
        data = json.loads(line)
        if 'bbh_' in data['data_source']:
            data['data_source'] = 'bbh'
        Baseline_model_data.append(data)
        

In [3]:
# dataset_name = 'full'
# folder_name = f'{dataset_name}_test_dataset'
from datasets import load_dataset
import pandas as pd
second_method_data = pd.read_parquet(adapThinkPath)

In [4]:
len(Baseline_model_data), len(second_method_data)

(2030, 460)

In [9]:
import numpy as np

accuracy_by_source = {}
for line in Baseline_model_data:
    if line['data_source'] not in accuracy_by_source:
        accuracy_by_source[line['data_source']] = []
    accuracy_by_source[line['data_source']].append(line['final_reward'])
    
for source in accuracy_by_source:
    mean_accuracy = np.mean(accuracy_by_source[source])
    print(source, mean_accuracy, len(accuracy_by_source[source]))
    
    

bbeh 0.20788177339901479 2030


In [11]:
import re
from math_verify.errors import TimeoutException
from math_verify.metric import math_metric
from math_verify.parser import ExprExtractionConfig, LatexExtractionConfig

def clean_prediction(model_output: str) -> str:
    """
    Try to normalize model outputs like 'Answer: C. Cotton' into '\boxed{C}'.
    """

    # If boxed answer is already there, keep it
    boxed_match = re.search(r'\\boxed\{([A-Za-z0-9]+)\}', model_output)
    if boxed_match:
        return f"\\boxed{{{boxed_match.group(1)}}}"

    # Otherwise, try to find a single-letter multiple-choice answer
    choice_match = re.search(r'\b([A-E])\b', model_output)
    if choice_match:
        return f"\\boxed{{{choice_match.group(1)}}}"

    # If nothing found, just return as-is
    return model_output.strip()

def getRawCorrectness(model_output: str, ground_truth: str) -> float:
    verify_func = math_metric(
        gold_extraction_target=(LatexExtractionConfig(),),
        pred_extraction_target=(ExprExtractionConfig(), LatexExtractionConfig()),
    )

    ret_score = 0.0
    ground_truth_boxed = f"\\boxed{{{ground_truth}}}"

    try:
        pred_clean = clean_prediction(model_output)
        ret_score, _ = verify_func([ground_truth_boxed], [pred_clean])
    except Exception:
        pass

    return ret_score

def getCorrectness(model_output: str, ground_truth: str) -> float:
    if '</think>' in model_output:
        model_output = model_output.split('</think>')[-1]

    verify_func = math_metric(
        gold_extraction_target=(LatexExtractionConfig(),),
        pred_extraction_target=(ExprExtractionConfig(), LatexExtractionConfig()),
    )

    ret_score = 0.0
    ground_truth_boxed = f"\\boxed{{{ground_truth}}}"

    try:
        pred_clean = clean_prediction(model_output)
        ret_score, _ = verify_func([ground_truth_boxed], [pred_clean])
    except Exception:
        pass

    return ret_score


In [12]:
import numpy as np
correctness_response_length_list_baseline = {}
from tqdm import tqdm
for line in tqdm(Baseline_model_data):
    if line['data_source'] not in correctness_response_length_list_baseline:
        correctness_response_length_list_baseline[line['data_source']] = []
    correctness_response_length_list_baseline[line['data_source']].append((getCorrectness(line['response'], line['ground_truth']), len(tokenizer.encode(line['response']))))
    
            
            
            

100%|██████████| 2030/2030 [01:05<00:00, 31.20it/s]


In [ ]:
filtered_dataset = []
average_score_data_source = {}
average_length_data_source = {}
correctness_response_length_list_second_method = {}
from tqdm import tqdm
for idx, row in tqdm(second_method_data.iterrows(), total=len(second_method_data), desc='Processing rows'):
    data_source = row['data_source']
    prompt = row['prompt']
    answer = row['reward_model']['ground_truth']

    if 'response' in row and row['response'] is not None:
        for res in row['response']:
            score = getCorrectness(res, answer)

            if data_source not in correctness_response_length_list_second_method:
                correctness_response_length_list_second_method[data_source] = []
            correctness_response_length_list_second_method[data_source].append((score, len(tokenizer.encode(res))))
            # filtered_dataset.append({
            #     'data_source': data_source,
            #     'prompt': prompt,
            #     'answer': answer,
            #     'response': res,
            #     'score': score,
            # })
            # if data_source not in average_score_data_source:
            #     average_score_data_source[data_source] = []
            # average_score_data_source[data_source].append(score)
            # if data_source not in average_length_data_source:
            #     average_length_data_source[data_source] = []
            # average_length_data_source[data_source].append(len(tokenizer.encode(res)))

Processing rows:   0%|          | 2/460 [00:00<00:31, 14.59it/s]We did not manage to extract a prediction in the correct format. Gold: ['\\boxed{carrot}'], Pred: ['brush']
We did not manage to extract a prediction in the correct format. Gold: ['\\boxed{4}'], Pred: ['No']
We did not manage to extract a prediction in the correct format. Gold: ['\\boxed{4}'], Pred: ['No']
Processing rows:   3%|▎         | 15/460 [00:01<00:36, 12.17it/s]We did not manage to extract a prediction in the correct format. Gold: ['\\boxed{Ambiguous}'], Pred: ["No.  \nLaurie received the free sample because she was already subscribed to the GC email list, which is one of the promotion's qualifying conditions. The fact that she did not change her subscription status is irrelevant to the eligibility criteria. The promotion is based on her existing subscription, not on whether she altered her status. Thus, the sample was received due to her subscription, not because of her unchanged status."]
We did not manage to ex

In [83]:
def evaluate_methods(baseline, new):
    """
    Compute relative accuracy and compression ratio between baseline and new method.

    Args:
        baseline (list[tuple]): (accuracy, length) for baseline method
        new (list[tuple]): (accuracy, length) for new method

    Returns:
        dict: results with relative accuracy and compression ratio
    """
    # Unpack
    base_acc = [a for a, _ in baseline]
    base_len = [l for _, l in baseline]
    new_acc = [a for a, _ in new]
    new_len = [l for _, l in new]

    # Average accuracy and lengths
    avg_base_acc = sum(base_acc) / len(base_acc)
    avg_new_acc = sum(new_acc) / len(new_acc)
    avg_base_len = sum(base_len) / len(base_len)
    avg_new_len = sum(new_len) / len(new_len)

    # Relative accuracy (% change)
    rel_acc = ((avg_new_acc - avg_base_acc) / avg_base_acc * 100
               if avg_base_acc != 0 else float('inf'))

    # Compression ratio (new/baseline)
    comp_ratio = avg_new_len / avg_base_len if avg_base_len != 0 else float('inf')

    # Percentage reduction in length
    comp_reduction = (1 - comp_ratio) * 100

    return {
        "Baseline Avg Accuracy": avg_base_acc,
        "New Avg Accuracy": avg_new_acc,
        "Relative Accuracy (%)": round(rel_acc, 2),
        "Baseline Avg Length": avg_base_len,
        "New Avg Length": avg_new_len,
        "Compression Ratio": round(comp_ratio, 4),
        "Compression Reduction (%)": round(comp_reduction, 2)
    }

In [84]:
for key in correctness_response_length_list_baseline:
    print("Data Source: ", key)
    compression = evaluate_methods(correctness_response_length_list_baseline[key], correctness_response_length_list_second_method[key])
    print("Baseline Model: ", compression["Baseline Avg Accuracy"])
    print("Second Method: ", compression["New Avg Accuracy"])
    print("Relative Accuracy (%):", compression["Relative Accuracy (%)"])
    print("Baseline Avg Length: ", compression["Baseline Avg Length"])
    print("New Avg Length: ", compression["New Avg Length"])
    print("Compression Ratio: ", compression["Compression Ratio"])
    print("Compression Reduction (%)", compression["Compression Reduction (%)"])
   
    print("--------------------------------")
    

Data Source:  bbeh
Baseline Model:  0.18275862068965518
Second Method:  0.20591133004926107
Relative Accuracy (%): 12.67
Baseline Avg Length:  6677.363546798029
New Avg Length:  4339.22512315271
Compression Ratio:  0.6498
Compression Reduction (%) 35.02
--------------------------------


In [7]:
def compute_aucoaa(data):
    # data is a list of (correctness, response_length) tuples
    n = len(data)
    tmax = max(length for _, length in data)
    oaa_t_list = []
    for t in range(tmax + 1):
        filtered = [correct for correct, length in data if length < t]
        if filtered:
            oaa_t = sum(filtered) / n  # Denominator is always n, per definition
        else:
            oaa_t = 0.0
        oaa_t_list.append(oaa_t)
    aucoaa = sum(oaa_t_list) / len(oaa_t_list)
    return aucoaa


In [8]:
for key in correctness_response_length_list.keys():
    print("Data Source: ", key)
    print("Accuracy: ", np.mean([correct for correct, _ in correctness_response_length_list[key]]))
    print("Response Length: ", np.mean([length for _, length in correctness_response_length_list[key]]))
    print("Number of Samples: ", len(correctness_response_length_list[key]))
    print("AUC-OAA: ", compute_aucoaa(correctness_response_length_list[key]))
    print("--------------------------------")

Data Source:  math-amc
Accuracy:  0.7951807228915663
Response Length:  4156.530120481928
Number of Samples:  415
AUC-OAA:  0.5376238678997872
--------------------------------
Data Source:  gpqa
Accuracy:  0.4720812182741117
Response Length:  4193.119796954315
Number of Samples:  985
AUC-OAA:  0.30289823809497224
--------------------------------
Data Source:  math-aime
Accuracy:  0.45454545454545453
Response Length:  6651.779545454546
Number of Samples:  440
AUC-OAA:  0.24861373632365236
--------------------------------


In [26]:
line['response']

'<think>\nOkay, so I need to figure out which of the given options A-J have the correct adjective order based on the example sentences provided. The user mentioned that in this variant of English, the adjectives follow a specific order. From the examples given, I need to determine what that order is.\n\nLooking at the example sentences, like "ancient key" (1), "pyramidal earring" (2), "repulsive normal-size hammer" (3), etc., I need to find a pattern in the adjectives. The correct adjective order in these examples probably follows a certain sequence, such as opinion, size, age, material, purpose, and then other descriptors.\n\nLet me try to deduce the order based on the examples. For instance, in (3) "repulsive normal-size hammer", the adjectives are "repulsive" (opinion), "normal-size" (size), and then "hammer" (noun). Similarly, (4) "red snorkeling bag" has "red" (color), "snorkeling" (purpose), then "bag". Wait, but that seems different. Wait, maybe the order is opinion, size, age, 

In [10]:
len(tokenizer.encode(line['response']))

4407

In [37]:
import json
## read jsonl
with open('your_file.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]


FileNotFoundError: [Errno 2] No such file or directory: 'your_file.jsonl'

In [1]:
from datasets import load_dataset
train_dataset = load_dataset('/nas-ssd2/joykirat/code/verl-fork/baselines/LC-R1/dataset', split='train')

Generating train split:   0%|          | 0/2499 [00:00<?, ? examples/s]

In [2]:
train_dataset

Dataset({
    features: ['problem', 'solution'],
    num_rows: 2499
})

In [3]:
train_dataset[0]

{'problem': 'Let $x$ , $y$ and $z$ all exceed $1$ and let $w$ be a positive number such that $\\log_xw=24$ , $\\log_y w = 40$ and $\\log_{xyz}w=12$ . Find $\\log_zw$ .',
 'solution': '60'}

In [9]:
## load parquet dataset
import pandas as pd
dataset = pd.read_parquet('/nas-ssd2/joykirat/code/verl-fork/verl/scripts/data/dapo-17k/train.parquet')
dataset.head()

,data_source,prompt,ability,reward_model,extra_info,uuid
0,DAPO-Math-17k,"[{'content': 'In triangle $ABC$, $\sin \angle ...",math,"{'ground_truth': '34', 'style': 'rule-lighteva...","{'index': 0, 'split': 'train'}",d2ccd328-e712-4d6b-b909-a7956589db4e
1,DAPO-Math-17k,[{'content': 'Let $ABCD$ be a unit square in t...,math,"{'ground_truth': '113', 'style': 'rule-lightev...","{'index': 1, 'split': 'train'}",5fb8b773-1e5f-48ab-b5ce-8d2ba6ed53cc
2,DAPO-Math-17k,"[{'content': 'Let $a, b, c$ be distinct number...",math,"{'ground_truth': '-3', 'style': 'rule-lighteva...","{'index': 2, 'split': 'train'}",974a3162-fb31-4720-86ab-6a3738ae84e7
3,DAPO-Math-17k,[{'content': 'There are $7$ boxes arranged in ...,math,"{'ground_truth': '3', 'style': 'rule-lighteval...","{'index': 3, 'split': 'train'}",c47003a1-29c7-46ad-b566-977ba9018acb
4,DAPO-Math-17k,[{'content': 'Let $S$ be the set of triples $(...,math,"{'ground_truth': '37', 'style': 'rule-lighteva...","{'index': 4, 'split': 'train'}",f8e041a5-2848-4dfb-99c4-74aa70c8d211


In [10]:
# convert this dataset into a dataset
from datasets import Dataset
dataset = Dataset.from_pandas(dataset)


In [12]:
dataset

Dataset({
    features: ['data_source', 'prompt', 'ability', 'reward_model', 'extra_info', 'uuid'],
    num_rows: 20000
})

In [13]:
dataset = load_dataset('parquet', data_files='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/data/dapo-17k/train.parquet')

Generating train split: 0 examples [00:00, ? examples/s]

In [17]:
### remove some columsn from the dataset
dataset = dataset.remove_columns(['data_source', 'ability', 'extra_info', 'uuid'])

In [21]:
dataset['train'][0]

{'prompt': [{'content': "In triangle $ABC$, $\\sin \\angle A = \\frac{4}{5}$ and $\\angle A < 90^\\circ$. Let $D$ be a point outside triangle $ABC$ such that $\\angle BAD = \\angle DAC$ and $\\angle BDC = 90^\\circ$. Suppose that $AD = 1$ and that $\\frac{BD}{CD} = \\frac{3}{2}$. If $AB + AC$ can be expressed in the form $\\frac{a\\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$. Let's think step by step and output the final answer within \\boxed{}.",
   'role': 'user'}],
 'reward_model': {'ground_truth': '34', 'style': 'rule-lighteval/MATH_v2'}}

In [22]:
dataset = dataset['train']

In [24]:
# create a new column with problem name from prompt
dataset = dataset.add_column('problem', [d['prompt'][0]['content'].replace("Let's think step by step and output the final answer within \\boxed{}.", '').strip() for d in dataset])


In [27]:
dataset = dataset.add_column('solution', [d['reward_model']['ground_truth'] for d in dataset])

In [29]:
dataset = dataset.remove_columns(['prompt', 'reward_model'])

In [30]:
dataset

Dataset({
    features: ['problem', 'solution'],
    num_rows: 20000
})

In [34]:
### save the dataset locally in dataset format '/nas-ssd2/joykirat/code/verl-fork/baselines/LC-R1/dapo-17k'
from datasets import load_dataset, load_from_disk
# dataset.save_to_disk('/nas-ssd2/joykirat/code/verl-fork/baselines/LC-R1/dapo-17k')

In [35]:
# load the dataset from the local dataset format
dataset = load_from_disk('/nas-ssd2/joykirat/code/verl-fork/baselines/LC-R1/dapo-17k')